In [112]:
import pandas as pd
import datetime
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pickle

In [52]:
train1 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
train = train[train.notnull().apply(all, axis=1)]
dev = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_dev.csv')
dev = dev[dev.notnull().apply(all, axis=1)]
test = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_test.csv')
test = test[test.notnull().apply(all, axis=1)]

dev_prepared = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrase_vectors_dev.csv', sep=';')
dev_prepared = dev_prepared.drop(columns='Unnamed: 0')
dev = dev.set_index('pairID')
y_hat = dev.loc[dev_prepared.iloc[:,0]].gold_label
dev_prepared = dev_prepared.iloc[:,1:].to_numpy()

In [118]:
def evaluate_performance(preds, actual):
    labels = ['contradiction', 'entailment', 'neutral']
    print('Confusion Matrix')
    print(confusion_matrix(actual, preds, labels=labels))
    print(f'F1-Score: {f1_score(actual, preds, labels=labels, average="macro")}')
    print(f'Accuracy: {accuracy_score(actual, preds)}')

In [209]:
def predict_y_from_z(z):
    if len(z.shape) > 1:
        z = pd.DataFrame(z)
        res = z.apply(predict_y_from_z, axis=1)
        return res.to_numpy()
    else:
        if all([z[i] == 'nan' or z[i] == 'neutral' for i in range(25)]):
            return 'neutral'
        elif any(z == 'contradiction'):
            return 'contradiction'
        else:
            return 'entailment'

In [138]:
directory = 'MLP_Classifiers_200k_training_10_iter_NN_size_200_200_50_30'
clf = list()
for i in range(25):
    with open(directory + "/MLP_Classifier" + str(i) + ".pkl", "rb") as f:
        clf += [pickle.load(f), ]

In [139]:
# Prepare colum indices
indices = [[0,1500], [0,1800], [0,2100], [0,2400], [0,2700],
           [300,1500], [300,1800], [300,2100], [300,2400], [300,2700],
           [600,1500], [600,1800], [600,2100], [600,2400], [600,2700],
           [900,1500], [900,1800], [900,2100], [900,2400], [900,2700],
           [1200,1500], [1200,1800], [1200,2100], [1200,2400], [1200,2700]]

# Initialise colulmn indices and "nan" values if information (e.g. location of sentence) is not detected
print(datetime.datetime.now())
print("Initialise column indices and 'nan' values")
not_nan = [None, ] * 25
cols = [None, ] * 25
for i in range(25):
    cols[i] = list(range(indices[i][0], indices[i][0]+300)) + list(range(indices[i][1],indices[i][1]+300))
    not_nan[i] = pd.Series([not x for x in pd.DataFrame(np.isnan(dev_prepared[:,cols[i]])).apply(any, axis=1)])
not_nan = np.array(not_nan).T

2023-02-27 15:35:47.435063
Initialise column indices and 'nan' values


In [210]:
z = np.empty((y_hat.shape[0], 25), dtype=np.dtype('U100'))

for i in range(25):
    z[not_nan[:,i], i] = clf[i].predict(dev_prepared[not_nan[:,i],:][:, cols[i]])

for j in range(y_hat.shape[0]):
    for i in range(25):
        if z[j,i] == '':
            z[j,i] = np.nan

y_hat_pred = predict_y_from_z(z)

In [211]:
y_hat_pred

array(['neutral', 'entailment', 'neutral', ..., 'entailment',
       'contradiction', 'neutral'], dtype=object)

In [175]:
z[0,3]

'nan'

In [154]:
y_hat_pred

array(['neutral', 'entailm', 'neutral', ..., 'entailm', 'contrad',
       'neutral'], dtype='<U7')

In [141]:
(y_hat_pred == 'neutral').mean()

0.2791173794358508

In [142]:
(y_hat == 'neutral').mean()

0.32745677888989994

In [212]:
evaluate_performance(y_hat_pred, y_hat)

Confusion Matrix
[[1945  572  448]
 [ 948 1050  950]
 [ 995  828 1056]]
F1-Score: 0.4508974464337985
Accuracy: 0.46075978161965425


In [148]:
y_hat_pred

array(['neutral', 'entailm', 'neutral', ..., 'entailm', 'contrad',
       'neutral'], dtype='<U7')

In [144]:
clf[0].predict(dev_prepared[not_nan[:,0],:][:, cols[0]])

array(['neutral', 'neutral', 'neutral', ..., 'neutral', 'neutral',
       'neutral'], dtype='<U13')